In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
# Import training data files

train_files = ["Data/fem1.csv", "Data/fem2.csv", "Data/fem3.csv", "Data/fem4.csv", "Data/fem5.csv",
               "Data/piros1.csv", "Data/piros2.csv", "Data/piros3.csv", "Data/piros4.csv", "Data/piros5.csv", "Data/piros6.csv"]

dt = 0.01  # timestep for numerical derivation

df_list = []
for filename in train_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    del df['NaN.2'], df['NaN.4'], df['NaN.5'], df['NaN.6'], df['NaN.7'], df['NaN.8'], df['NaN.9']
    df['RelErr'] = np.abs((df['NaN.11'] - df['NaN.10'])/df['NaN.11'])
    del df['NaN.10'], df['NaN.11']
    df.rename(columns = {'NaN':'Time', 'NaN.1': 'Psi', 'NaN.3': 'Mz'}, inplace = True)
    df['Ome'] = (df['Psi'].shift(periods=1) - df['Psi'])/dt
    df['Eps'] = (df['Ome'].shift(periods=1) - df['Ome'])/dt
    df_list.append(df)

df_train = pd.concat(df_list, axis=0, ignore_index=True)
df_train = df_train.dropna(axis=0)
df_train.head()

In [ ]:
# I/O normalization based on the training data set

df_normed = (df_train - df_train.mean())/df_train.std()  # normalization
df_normed = df_normed.sample(frac=1)  # shuffles the data
del df_normed['Time']  # time is not needed
df_normed.head()

In [ ]:
target = df_normed.pop('RelErr') # relative error (target of prediction)
tf.convert_to_tensor(df_normed) # convert training data to tensor

In [ ]:
def get_basic_model(n_hidden=2, n_nodes=64, reg_lambda=0.001):
    regularizer = tf.keras.regularizers.l2(reg_lambda) # L2 (Ridge) regularization
    
    model = tf.keras.Sequential() # sequential keras model
    
    # input layer
    model.add(tf.keras.layers.Dense(n_nodes, input_shape=(4,), activation='tanh',
                                    kernel_regularizer=regularizer))
    
    # n number of hidden layers
    for i in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_nodes, activation='tanh', kernel_regularizer=regularizer))
        
    # output layer
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss='mean_squared_error', # MSE
                  metrics = [tf.keras.metrics.RootMeanSquaredError()] # RMSE
                  )
    return model

In [ ]:
n_layers = 1  # number of hidden layers
n_nodes = 16  # number of nodes per layer

model = get_basic_model(n_hidden=n_layers, n_nodes=n_nodes , reg_lambda=0.001)

EPOCHS = 5000
BATCH_SIZE = 512

# early stopper only to call back the model with the best validation error
early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = EPOCHS, mode = "min",
                                                 restore_best_weights = True)

history = model.fit(
    df_normed, target,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    validation_split = 0.2,
    verbose = 1,
    shuffle = False,  # do not shuffle, because training data is already mixed
    callbacks=[early_stopper]
)

In [ ]:
# Training and validation losses

plt.figure(figsize=(4,3), layout="constrained")
plt.plot(history.history['root_mean_squared_error'], 'k')
plt.plot(history.history['val_root_mean_squared_error'], 'r')
plt.ylabel('RMSE loss (normalized) [1]')
plt.xlabel('Epochs [1]')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.xlim((0, EPOCHS))
plt.grid()
plt.savefig(f"TrainingErrors_hl{n_layers}_npl{n_nodes}.pdf", facecolor="white")
plt.show()

In [ ]:
# Loading test data

test_files = ["Data/fem6.csv", "Data/piros7.csv"]

dt = 0.01  # timestep for numerical derivation

df_list = []
for filename in test_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    del df['NaN.2'], df['NaN.4'], df['NaN.5'], df['NaN.6'], df['NaN.7'], df['NaN.8'], df['NaN.9']
    df['RelErr'] = np.abs((df['NaN.11'] - df['NaN.10'])/df['NaN.11'])
    del df['NaN.10'], df['NaN.11']
    df.rename(columns = {'NaN':'Time', 'NaN.1': 'Psi', 'NaN.3': 'Mz'}, inplace = True)
    df['Ome'] = (df['Psi'].shift(periods=1) - df['Psi'])/dt
    df['Eps'] = (df['Ome'].shift(periods=1) - df['Ome'])/dt
    df_list.append(df)

df_test = pd.concat(df_list, axis=0, ignore_index=True)
df_test = df_test.dropna(axis=0)
df_test.head()

In [ ]:
# Standardization (based on training data)

df_test_normed = (df_test - df_train.mean())/df_train.std()
del df_test_normed['Time']  # delete time data
df_test_normed.head()

In [ ]:
# Model evaluation

test_target = df_test_normed.pop('RelErr').to_numpy()  # target of prediction
tf.convert_to_tensor(df_test_normed)

test_results = model.predict(df_test_normed)

In [ ]:
# Back scaling
test_target = test_target*df_train['RelErr'].std() + df_train['RelErr'].mean()
test_results = test_results*df_train['RelErr'].std() + df_train['RelErr'].mean()

# Error of prediction
RMSE = np.sqrt(np.mean((test_target - test_results)**2))
RMSE

In [ ]:
# Illustrating the results

plt.figure(figsize=(4.5,3), layout="constrained")
plt.plot(test_target, 'k')
plt.plot(test_results, '--r')
plt.xlabel("Test data index")
plt.ylabel("Absolute relative error [1]")
plt.legend(("Measured error", "Predicted error"))
plt.xlim((0, test_target.shape[0]))
plt.grid()
plt.savefig(f'TestResults_hl{n_layers}_npl{n_nodes}.pdf', facecolor="white")
plt.show()